In [1]:
# import requisite module
import pandas as pd 
import json
import seaborn as sns
import numpy as np

In [2]:
# read data from json 
json_data = pd.read_json('https://data.sfgov.org/resource/fjjd-jecq.json?$limit=2000000')

Right now SODA API has a limit of returning 1000 rows at a time when querying the dataset. To query more than 1000 rows, I added '$limit=' parameter to json url which will set a limit on how much I want to query from a dataset. 

In [48]:
# save data in pandas dataframe
calls_for_service=pd.DataFrame(json_data)

In [49]:
calls_for_service.shape

(2000000, 14)

In [50]:
calls_for_service.head()

,address,address_type,agency_id,call_date,call_dttm,call_time,city,common_location,crime_id,disposition,offense_date,original_crimetype_name,report_date,state
0,1500 Block Of Pine St,Premise Address,1,2016-09-20T00:00:00.000,2016-09-20T11:50:00.000,2018-11-08 11:50:00,San Francisco,NaN,162641608,REP,2016-09-20T00:00:00.000,Complaint Unkn,2016-09-20T00:00:00.000,CA
1,100 Block Of Erie St,Premise Address,1,2016-09-20T00:00:00.000,2016-09-20T12:36:00.000,2018-11-08 12:36:00,San Francisco,NaN,162641785,UTL,2016-09-20T00:00:00.000,909,2016-09-20T00:00:00.000,CA
2,900 Block Of Market St,Premise Address,1,2016-09-20T00:00:00.000,2016-09-20T14:01:00.000,2018-11-08 14:01:00,San Francisco,NaN,162642180,HAN,2016-09-20T00:00:00.000,Burglary,2016-09-20T00:00:00.000,CA
3,1900 Block Of Palou Av,Premise Address,1,2016-09-20T00:00:00.000,2016-09-20T14:30:00.000,2018-11-08 14:30:00,San Francisco,NaN,162642293,REP,2016-09-20T00:00:00.000,Burglary,2016-09-20T00:00:00.000,CA
4,Florida St/division St,Intersection,1,2016-09-20T00:00:00.000,2016-09-20T14:49:00.000,2018-11-08 14:49:00,San Francisco,NaN,162642379,HAN,2016-09-20T00:00:00.000,Encampment,2016-09-20T00:00:00.000,CA


Dataset has 2 million rows and 14 columns

In [51]:
calls_for_service.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 14 columns):
address                    2000000 non-null object
address_type               2000000 non-null object
agency_id                  2000000 non-null int64
call_date                  2000000 non-null object
call_dttm                  2000000 non-null object
call_time                  2000000 non-null datetime64[ns]
city                       1944403 non-null object
common_location            197653 non-null object
crime_id                   2000000 non-null int64
disposition                2000000 non-null object
offense_date               2000000 non-null object
original_crimetype_name    2000000 non-null object
report_date                2000000 non-null object
state                      2000000 non-null object
dtypes: datetime64[ns](1), int64(2), object(11)
memory usage: 213.6+ MB


In [52]:
#check for the null entries 
calls_for_service.isnull().sum()

address                          0
address_type                     0
agency_id                        0
call_date                        0
call_dttm                        0
call_time                        0
city                         55597
common_location            1802347
crime_id                         0
disposition                      0
offense_date                     0
original_crimetype_name          0
report_date                      0
state                            0
dtype: int64

In [53]:
# drop column common_location as it has many null entries
calls_for_service = calls_for_service.drop('common_location', axis=1)

In [55]:
# forward fill city column values to replace the null values 
calls_for_service['city'] = calls_for_service['city'].bfill()

In [56]:
calls_for_service.isnull().sum()


address                    0
address_type               0
agency_id                  0
call_date                  0
call_dttm                  0
call_time                  0
city                       0
crime_id                   0
disposition                0
offense_date               0
original_crimetype_name    0
report_date                0
state                      0
dtype: int64

Observe that there are no NULL values in city column anymore.

In [57]:
# change the datatype of call_dttm column
calls_for_service['call_dttm'] = pd.to_datetime(calls_for_service['call_dttm'])
#drop call_date and call_time column as dataset already has a call_dttm column, also drop agency_id
calls_for_service = calls_for_service.drop(['call_time','call_date','agency_id'], axis=1)

In [58]:
calls_for_service.shape

(2000000, 10)

In [59]:
# Remove the white spaces,special characters from the original crime types values
calls_for_service['original_crimetype_name'].str.strip()
calls_for_service['original_crimetype_name'] = calls_for_service['original_crimetype_name'].map(lambda x: x.lstrip('e`&[***~ +-".,//\\0').rstrip(' ***."'))

# Replace misspelled original crime types with correct values
calls_for_service['original_crimetype_name'] = calls_for_service['original_crimetype_name'].replace(to_replace=['Yelinng','Yelling Male','Yeller','Yelling Man','Yelling Off Balcony','Yelling/aggressive'], value='Yelling', regex=True)
calls_for_service['original_crimetype_name'] = calls_for_service['original_crimetype_name'].replace(to_replace=['Suspicious person'], value='Suspicious Person', regex=True)
calls_for_service['original_crimetype_name'] = calls_for_service['original_crimetype_name'].replace(to_replace=['Yel Zone','Yel Zn','Yel Grn Zn','Yz','Yello Zone','Yellow Zne','Yz,gz,rz','Yzone','Yellow Zone,gz,rz','Yellow Zoneone'], value='Yellow Zone', regex=True)
calls_for_service.loc[calls_for_service['original_crimetype_name'].str.contains('152'), 'original_crimetype_name'] = '152'
calls_for_service.loc[calls_for_service['original_crimetype_name'].str.contains('1030'), 'original_crimetype_name'] = '10-30'
calls_for_service.loc[calls_for_service['original_crimetype_name'].str.contains('207'), 'original_crimetype_name'] = '207'
calls_for_service.loc[calls_for_service['original_crimetype_name'].str.contains('212'), 'original_crimetype_name'] = '212'

# Count the occurence of each crime type
data_by_city_crime=calls_for_service.groupby(['original_crimetype_name']).size().reset_index(name='count')
data_by_city_crime

,original_crimetype_name,count
0,,41
1,'Fire Hazard',1
2,'S,2
3,(221 Taken),1
4,(459),1
5,(487),1
6,(Nom 851),1
7,1,1
8,1 At Risk,1
9,1 Yo,1


In [60]:
# Replace Null values of original crimetype with Nan and then drop the rows having nan
calls_for_service['original_crimetype_name'].replace('', np.nan, inplace=True)
calls_for_service.dropna(subset=['original_crimetype_name'], inplace=True)
calls_for_service.shape

(1999959, 10)

In [61]:
# read another exel file which has radio codes 
radio_code_xl = pd.read_excel("Radio_Codes_2016.xlsx")
radio_code = pd.DataFrame(radio_code_xl)
radio_code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 2 columns):
original_crimetype_name    178 non-null object
Meaning                    178 non-null object
dtypes: object(2)
memory usage: 2.9+ KB


In [62]:
# replace radio codes mentioned in original_crimetype_name column with radio codes meaning from radio code data frame
calls_for_service['original_crimetype_name'] = calls_for_service['original_crimetype_name'].astype(str)
radio_code['original_crimetype_name']= radio_code['original_crimetype_name'].astype(str)
radio_code['Meaning']= radio_code['Meaning'].astype(str)
calls_for_service['original_crimetype_name'] = calls_for_service['original_crimetype_name'].map(radio_code.set_index('original_crimetype_name')['Meaning']).fillna(calls_for_service['original_crimetype_name'])

In [63]:
calls_for_service.head()

,address,address_type,call_dttm,city,crime_id,disposition,offense_date,original_crimetype_name,report_date,state
0,1500 Block Of Pine St,Premise Address,2016-09-20 11:50:00,San Francisco,162641608,REP,2016-09-20T00:00:00.000,Complaint Unkn,2016-09-20T00:00:00.000,CA
1,100 Block Of Erie St,Premise Address,2016-09-20 12:36:00,San Francisco,162641785,UTL,2016-09-20T00:00:00.000,Interview a citizen,2016-09-20T00:00:00.000,CA
2,900 Block Of Market St,Premise Address,2016-09-20 14:01:00,San Francisco,162642180,HAN,2016-09-20T00:00:00.000,Burglary,2016-09-20T00:00:00.000,CA
3,1900 Block Of Palou Av,Premise Address,2016-09-20 14:30:00,San Francisco,162642293,REP,2016-09-20T00:00:00.000,Burglary,2016-09-20T00:00:00.000,CA
4,Florida St/division St,Intersection,2016-09-20 14:49:00,San Francisco,162642379,HAN,2016-09-20T00:00:00.000,Encampment,2016-09-20T00:00:00.000,CA


In [65]:
data_by_city_crime[data_by_city_crime['count']== 1]

,original_crimetype_name,count
2,'Fire Hazard',1
4,(221 Taken),1
5,(459),1
6,(487),1
7,(Nom 851),1
8,1,1
9,1 At Risk,1
10,1 Yo,1
11,1-Man Band,1
14,10 Smoking Weed,1


In [21]:
# Drop rows of original crime type which appeared once
calls_for_service.loc[calls_for_service.groupby('original_crimetype_name').original_crimetype_name.transform('count').lt(2),'original_crimetype_name']= np.nan
calls_for_service.dropna(subset=['original_crimetype_name'], inplace=True)

In [37]:
calls_for_service.groupby(['original_crimetype_name']).size().reset_index(name='count')

,original_crimetype_name,count
0,'S,2
1,10 Ago,2
2,10 Min Ago,3
3,10-30 Poss,2
4,100a,25
5,100p Veh,2
6,100s,6
7,100v,17
8,1036,25
9,123,122


In [66]:
# count the crimes per city
data_by_city=calls_for_service.groupby(['city']).size().reset_index(name='count')
data_by_city

,city,count
0,Brisbane,38
1,Daly City,1001
2,Fort Mason,67
3,Hunters Point,573
4,Presidio,526
5,San Francisco,1986600
6,Treasure Isla,9927
7,Yerba Buena,1227


In [70]:
# display the top 25 crimes 
data_by_city_crime.sort_values('count',ascending=False).head(50)

,original_crimetype_name,count
11069,Passing Call,260977
14972,Traffic Stop,210708
14602,Suspicious Person,104815
8774,Homeless Complaint,102769
409,22500e,76454
10325,Muni Inspection,68826
5476,Audible Alarm,58056
15072,Trespasser,52634
16222,Well Being Check,47954
14604,Suspicious Vehicle,47448
